In [ ]:
%tensorflow_version 2.x
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
!pip install imageio==2.4.1
!pip install imgaug==0.2.6
!pip uninstall opencv-python -y
!pip install opencv-python==4.1.2.30
import cv2
!pip install lucid==0.2.3
!pip install moviepy

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.append('./detectron2/projects/DensePose')

In [ ]:
!python detectron2/tests/test_model_analysis.py

In [ ]:
!mkdir /content/temp
# TODO: upload your video here
# !wget http://gaze360.csail.mit.edu/files/video.mp4
!ffmpeg -i VCP107SE.mp4 /content/temp/frame%04d.jpg 
!rm -r /content/DensePose/DensePoseData/infer_out/*

In [ ]:
!cd detectron2/projects/DensePose && wget https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_101_FPN_s1x/165712084/model_final_c6ab63.pkl

In [ ]:
pip install av

In [ ]:
!cd detectron2/projects/DensePose && python apply_net.py show configs/densepose_rcnn_R_101_FPN_s1x.yaml model_final_c6ab63.pkl /content/temp/frame0001.jpg dp_contour,bbox --output image_densepose_contour.png

In [ ]:
!mkdir DensePoseData
!cd detectron2/projects/DensePose && python apply_net.py \
  dump \
  configs/densepose_rcnn_R_101_FPN_s1x.yaml model_final_c6ab63.pkl \
  /content/temp \
  -v \
  --output /content/DensePoseData/data.pkl

In [ ]:
with Path('./DensePoseData/data.pkl','rb') as fid:
  data = pickle.load(fid)


In [ ]:
!pip install scikit-video
import torch
import imageio
import skvideo.io  
import imageio
import cv2
import numpy as np
import os
from pathlib import Path
import pickle
import densepose

v_reader = skvideo.io.vread("WB033ESF.mp4") 
v = list(v_reader)



f = open('./DensePoseData/data.pkl','rb')
data = torch.load(f,map_location='cpu')




In [ ]:
import imageio
import cv2
import numpy as np
import os
from pathlib import Path
import pickle
import densepose

v_reader = imageio.get_reader('WB033ESF.mp4')
v = list(v_reader)

with Path('./DensePoseData/data.pkl').open('rb') as fid:
  data = pickle.load(fid)



In [ ]:
import numpy as np


In [ ]:
def get_head_bbox(body_bbox, dense_pose_record):
    #Code from Detectron modifyied to make the box 30% bigger
    """Compute the tight bounding box of a binary mask."""
    # Head mask
    labels = dense_pose_record.labels.cpu().numpy()
    mask = (labels == 23) | (labels == 24)    
    

    if not np.any(mask):
      return None

    xs = np.where(np.sum(mask, axis=0) > 0)[0]
    ys = np.where(np.sum(mask, axis=1) > 0)[0]

    W = labels.shape[0]
    H = labels.shape[1]
    
    if len(xs) == 0 or len(ys) == 0:
        return None

    x0 = xs[0]
    x1 = xs[-1]

    y0 = ys[0]
    y1 = ys[-1]
    w = x1-x0
    h = y1-y0
    
    x0 = max(0,x0-w*0.15)
    x1 = max(0,x1+w*0.15)
    y0 = max(0,y0-h*0.15)
    y1 = max(0,y1+h*0.15)

    x0 += body_bbox[0]
    x1 += body_bbox[0]
    y0 += body_bbox[1]
    y1 += body_bbox[1]

    return np.array((x0, y0, x1, y1), dtype=np.float32)


def extract_heads_bbox(frame):
    N = len(frame['pred_densepose'])
    if len(frame['pred_densepose'])==0:
      return []
    bbox_list = []
    for i in range(len(frame['pred_densepose'])-1,-1,-1):
      dp_record = frame['pred_densepose'][i]
      body_bbox = frame['pred_boxes_XYXY'][i].cpu().numpy()
      bbox = get_head_bbox(body_bbox, dp_record)
      if bbox is None: 
        continue
      bbox_list.append(bbox)
    return bbox_list

names = np.array([int(Path(frame['file_name']).stem[-4:]) - 1 for frame in data], object)

bboxes = np.array([extract_heads_bbox(frame) for frame in data], object)
final_results = {n: b for n, b in zip(names, bboxes) if len(bboxes) > 0} 
#final_results
#df = pd.DataFrame(con)
#df.to_csv('myfile.csv')

#print(data[0]['pred_densepose'][0])
#print(data[0]['pred_boxes_XYXY'][0])
#print(bboxes[0][0])
#imageio.imwrite('test.png', (data[100]['pred_densepose'][1].labels.cpu().numpy() == 23).astype(float))

In [ ]:
def compute_iou(bb1,bb2):
    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[2]-bb1[0]) * (bb1[3]-bb1[1])
    bb2_area = (bb2[2]-bb2[0]) * (bb2[3]-bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    eps = 1e-8

    if iou <= 0.0  or iou > 1.0 + eps: return 0.0

    return iou
    
def find_id(bbox,id_dict):
    id_final = None
    max_iou = 0.5
    for k in id_dict.keys():
        if(compute_iou(bbox,id_dict[k][0])>max_iou): 
            id_final = k
            max_iou = compute_iou(bbox,id_dict[k][0])
    return id_final


id_num = 0
tracking_id = dict()
identity_last = dict()
frames_with_people = list(final_results.keys())

frames_with_people.sort()
for i in frames_with_people:
    speople = final_results[i]
    identity_next = dict()
    for j in range(len(speople)):
        bbox_head = speople[j]
        if bbox_head is None: continue
        id_val = find_id(bbox_head,identity_last)
        if id_val is None: 
            id_num+=1
            id_val = id_num
        #TODO: Improve eye location
        eyes = [(bbox_head[0]+bbox_head[2])/2.0, (0.65*bbox_head[1]+0.35*bbox_head[3])]
        identity_next[id_val] = (bbox_head,eyes)
        final_results = {n: b for n, b in zip(names, bboxes) if len(bboxes) > 0} 

    identity_last = identity_next
    tracking_id[i] = identity_last

print(tracking_id)
#print(final_results)


{0: {1: (array([298.70474, 740.91406, 467.75473, 882.3641 ], dtype=float32), [383.229736328125, 790.4215667724609]), 2: (array([1148.3799,  655.981 , 1327.7798,  855.431 ], dtype=float32), [1238.079833984375, 725.7885223388671]), 3: (array([793.36755, 459.62607, 962.36755, 684.6261 ], dtype=float32), [877.8675537109375, 538.3760787963868])}, 1: {1: (array([300.72668, 738.8313 , 466.3267 , 882.2813 ], dtype=float32), [383.52667236328125, 789.0388031005859]), 2: (array([1147.3285,  654.8386, 1328.0284,  855.1386], dtype=float32), [1237.678466796875, 724.9436187744141]), 3: (array([794.68506, 458.52368, 961.0851 , 685.6737 ], dtype=float32), [877.8850708007812, 538.0261901855469])}, 2: {1: (array([300.3822 , 738.7654 , 465.9822 , 883.36536], dtype=float32), [383.18218994140625, 789.3753723144531]), 2: (array([1148.2944 ,  656.57526, 1328.9944 ,  854.87524], dtype=float32), [1238.6444091796875, 725.9802520751953]), 3: (array([793.5278, 457.8404, 959.9278, 684.9904], dtype=float32), [876.72

In [ ]:
from collections import defaultdict
from collections import OrderedDict
"""
person1:
 {
   frame1: List[int] of length 4
 }
"""
persondict = defaultdict(OrderedDict)
for frame, pinfo in tracking_id.items():
  for person, values in pinfo.items():
      persondict[person][frame] = list(values[0])
#print(persondict.keys())
print(persondict[5])



OrderedDict([(54, [1122.8347, 779.3371, 1203.4348, 824.1871])])


In [ ]:
for person, values in persondict.items():
  f = open(f'person_{person}.txt', "w")
  for frame, bbox in values.items():
    frame_string = str(frame+1).zfill(4)
    coordstring = ",".join([str(int(c)) for c in bbox])
    f.write(f'{frame_string}.jpg,{coordstring}\n')
  f.close()

In [ ]:
! git clone https://github.com/ejcgt/attention-target-detection.git


fatal: destination path 'attention-target-detection' already exists and is not an empty directory.


In [ ]:
!pip install scipy==1.1.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mv "attention-target-detection/utils" "utils"
!mv "attention-target-detection/lib" "lib"

!mv "attention-target-detection/misc" "misc"
!mv "attention-target-detection/config.py" "config.py"
!mv "attention-target-detection/dataset.py" "dataset.py"
!mv "attention-target-detection/demo.py" "demo.py"
!mv "attention-target-detection/download_models.sh" "download_models.sh"
!mv "attention-target-detection/environment.yml" "environment.yml"
!mv "attention-target-detection/eval_on_gazefollow.py" "eval_on_gazefollow.py"
!mv "attention-target-detection/eval_on_videoatttarget.py" "eval_on_videoatttarget.py"
!mv "attention-target-detection/model.py" "model.py"
!mv "attention-target-detection/train_on_gazefollow.py" "train_on_gazefollow.py"
!mv "attention-target-detection/train_on_videoatttarget.py" "train_on_videoatttarget.py"





In [ ]:
!sh download_models.sh


--2022-11-10 05:03:45--  https://www.dropbox.com/s/s9y65ajzjz4thve/initial_weights_for_spatial_training.pt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/s9y65ajzjz4thve/initial_weights_for_spatial_training.pt [following]
--2022-11-10 05:03:46--  https://www.dropbox.com/s/raw/s9y65ajzjz4thve/initial_weights_for_spatial_training.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf237ef40e01e31a957f6bec2c6.dl.dropboxusercontent.com/cd/0/inline/Bwe1O2jnCZW3V8B7k-w065twL_u8Yfp_3VzA_dFafEaipEkD9x2ZMNmUFxtappaEYgqD3rO4KBaA4gNvGsYUaMcUjDIhVgzKYfB8aJKf446CpPC0RQ5mHGi6sw2J1bAV2i1YosfILtrmJnzcifZp-shky8oLa0FjhbOEbE9ii1yk_w/file# [following]
--2022-11-10 05:03:46--  https://ucf237ef40e01e31a957f6bec2c6.dl.dropboxusercontent.com/cd/0/inl

Here you need to create a folder data and has the structure like this: data/demo/frames




In [ ]:
!mkdir /content
!ffmpeg -i WB033ESF.mp4 /content/data/demo/frames/%04d.jpg 


mkdir: cannot create directory ‘/content’: File exists
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enabl

In [ ]:
!mv "person_11.txt" "data/demo/person1.txt"




In [ ]:
import argparse, os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from skimage.transform import resize

from model import ModelSpatial
from utils import imutils, evaluation
from config import *

parser = argparse.ArgumentParser()
parser.add_argument('--model_weights', type=str, help='model weights', default='model_demo.pt')
parser.add_argument('--image_dir', type=str, help='images', default='data/demo/frames')
parser.add_argument('--head', type=str, help='head bounding boxes', default='data/demo/person1.txt')
parser.add_argument('--vis_mode', type=str, help='heatmap or arrow', default='arrow')
parser.add_argument('--out_threshold', type=int, help='out-of-frame target dicision threshold', default=100)
args, unknown = parser.parse_known_args()

In [ ]:
def _get_transform():
    transform_list = []
    transform_list.append(transforms.Resize((input_resolution, input_resolution)))
    transform_list.append(transforms.ToTensor())
    transform_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return transforms.Compose(transform_list)


def run():
    column_names = ['frame', 'left', 'top', 'right', 'bottom']
    df = pd.read_csv(args.head, names=column_names, index_col=0)
    df['left'] -= (df['right']-df['left'])*0.1
    df['right'] += (df['right']-df['left'])*0.1
    df['top'] -= (df['bottom']-df['top'])*0.1
    df['bottom'] += (df['bottom']-df['top'])*0.1

    # set up data transformation
    test_transforms = _get_transform()

    model = ModelSpatial()
    model_dict = model.state_dict()
    pretrained_dict = torch.load(args.model_weights)
    pretrained_dict = pretrained_dict['model']
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    model.cuda()
    model.train(False)
    df2=pd.DataFrame(columns=['x', 'y'])

    with torch.no_grad():
        for i in df.index:
            frame_raw = Image.open(os.path.join(args.image_dir, i))
            frame_raw = frame_raw.convert('RGB')
            width, height = frame_raw.size

            head_box = [df.loc[i,'left'], df.loc[i,'top'], df.loc[i,'right'], df.loc[i,'bottom']]

            head = frame_raw.crop((head_box)) # head crop

            head = test_transforms(head) # transform inputs
            frame = test_transforms(frame_raw)
            head_channel = imutils.get_head_box_channel(head_box[0], head_box[1], head_box[2], head_box[3], width, height,
                                                        resolution=input_resolution).unsqueeze(0)

            head = head.unsqueeze(0).cuda()
            frame = frame.unsqueeze(0).cuda()
            head_channel = head_channel.unsqueeze(0).cuda()

            # forward pass
            raw_hm, _, inout = model(frame, head_channel, head)

            # heatmap modulation
            raw_hm = raw_hm.cpu().detach().numpy() * 255
            raw_hm = raw_hm.squeeze()
            inout = inout.cpu().detach().numpy()
            inout = 1 / (1 + np.exp(-inout))
            inout = (1 - inout) * 255
            norm_map = resize(raw_hm, (height, width)) - inout

            # vis
            plt.close()
            fig = plt.figure()
            #fig.canvas.manager.window.move(0,0)
            plt.axis('off')
            plt.imshow(frame_raw)

            ax = plt.gca()
            rect = patches.Rectangle((head_box[0], head_box[1]), head_box[2]-head_box[0], head_box[3]-head_box[1], linewidth=2, edgecolor=(0,1,0), facecolor='none')
            ax.add_patch(rect)

            pred_x, pred_y = evaluation.argmax_pts(raw_hm)
            norm_p = [pred_x/output_resolution, pred_y/output_resolution]
            x, y = norm_p[0]*width, norm_p[1]*height
            row = {'x': x, 'y': y}
            #df2=df2.append(row, ignore_index=True)
            
            if args.vis_mode == 'arrow':
                if inout < args.out_threshold: # in-frame gaze
                    pred_x, pred_y = evaluation.argmax_pts(raw_hm)

                    norm_p = [pred_x/output_resolution, pred_y/output_resolution]
                    circ = patches.Circle((norm_p[0]*width, norm_p[1]*height), height/50.0, facecolor=(1,1,1), edgecolor='none')
                    ax.add_patch(circ)
                    plt.plot((norm_p[0]*width,(head_box[0]+head_box[2])/2), (norm_p[1]*height,(head_box[1]+head_box[3])/2), '-', color=(0,1,0,1))
            else:
                plt.imshow(norm_map, cmap = 'jet', alpha=0.2, vmin=0, vmax=255)
            

            plt.show(block=False)
            plt.pause(0.2)

        print('DONE!')
        #df2.to_csv('person1.csv')
if __name__ == "__main__":
    run()


In [ ]:
import argparse, os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from skimage.transform import resize

from model import ModelSpatial
from utils import imutils, evaluation
from config import *

parser = argparse.ArgumentParser()
parser.add_argument('--model_weights', type=str, help='model weights', default='model_demo.pt')
parser.add_argument('--image_dir', type=str, help='images', default='data/demo/frames')
parser.add_argument('--head', type=str, help='head bounding boxes', default='data/demo/person2.txt')
parser.add_argument('--vis_mode', type=str, help='heatmap or arrow', default='arrow')
parser.add_argument('--out_threshold', type=int, help='out-of-frame target dicision threshold', default=100)
args, unknown = parser.parse_known_args()

In [ ]:
def _get_transform():
    transform_list = []
    transform_list.append(transforms.Resize((input_resolution, input_resolution)))
    transform_list.append(transforms.ToTensor())
    transform_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return transforms.Compose(transform_list)


def run():
    column_names = ['frame', 'left', 'top', 'right', 'bottom']
    df = pd.read_csv(args.head, names=column_names, index_col=0)
    df['left'] -= (df['right']-df['left'])*0.1
    df['right'] += (df['right']-df['left'])*0.1
    df['top'] -= (df['bottom']-df['top'])*0.1
    df['bottom'] += (df['bottom']-df['top'])*0.1

    # set up data transformation
    test_transforms = _get_transform()

    model = ModelSpatial()
    model_dict = model.state_dict()
    pretrained_dict = torch.load(args.model_weights)
    pretrained_dict = pretrained_dict['model']
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    model.cuda()
    model.train(False)
    df2=pd.DataFrame(columns=['x', 'y'])

    with torch.no_grad():
        for i in df.index:
            frame_raw = Image.open(os.path.join(args.image_dir, i))
            frame_raw = frame_raw.convert('RGB')
            width, height = frame_raw.size

            head_box = [df.loc[i,'left'], df.loc[i,'top'], df.loc[i,'right'], df.loc[i,'bottom']]

            head = frame_raw.crop((head_box)) # head crop

            head = test_transforms(head) # transform inputs
            frame = test_transforms(frame_raw)
            head_channel = imutils.get_head_box_channel(head_box[0], head_box[1], head_box[2], head_box[3], width, height,
                                                        resolution=input_resolution).unsqueeze(0)

            head = head.unsqueeze(0).cuda()
            frame = frame.unsqueeze(0).cuda()
            head_channel = head_channel.unsqueeze(0).cuda()

            # forward pass
            raw_hm, _, inout = model(frame, head_channel, head)

            # heatmap modulation
            raw_hm = raw_hm.cpu().detach().numpy() * 255
            raw_hm = raw_hm.squeeze()
            inout = inout.cpu().detach().numpy()
            inout = 1 / (1 + np.exp(-inout))
            inout = (1 - inout) * 255
            norm_map = resize(raw_hm, (height, width)) - inout

            # vis
            plt.close()
            fig = plt.figure()
            #fig.canvas.manager.window.move(0,0)
            plt.axis('off')
            plt.imshow(frame_raw)

            ax = plt.gca()
            rect = patches.Rectangle((head_box[0], head_box[1]), head_box[2]-head_box[0], head_box[3]-head_box[1], linewidth=2, edgecolor=(0,1,0), facecolor='none')
            ax.add_patch(rect)

            pred_x, pred_y = evaluation.argmax_pts(raw_hm)
            norm_p = [pred_x/output_resolution, pred_y/output_resolution]
            x, y = norm_p[0]*width, norm_p[1]*height
            row = {'x': x, 'y': y}
            #df2=df2.append(row, ignore_index=True)
            
            if args.vis_mode == 'arrow':
                if inout < args.out_threshold: # in-frame gaze
                    pred_x, pred_y = evaluation.argmax_pts(raw_hm)

                    norm_p = [pred_x/output_resolution, pred_y/output_resolution]
                    circ = patches.Circle((norm_p[0]*width, norm_p[1]*height), height/50.0, facecolor=(1,1,1), edgecolor='none')
                    ax.add_patch(circ)
                    plt.plot((norm_p[0]*width,(head_box[0]+head_box[2])/2), (norm_p[1]*height,(head_box[1]+head_box[3])/2), '-', color=(0,1,0,1))
            else:
                plt.imshow(norm_map, cmap = 'jet', alpha=0.2, vmin=0, vmax=255)
            

            plt.show(block=False)
            plt.pause(0.2)

        print('DONE!')
        df2.to_csv('person2.csv')
if __name__ == "__main__":
    run()


In [ ]:
def _get_transform():
    transform_list = []
    transform_list.append(transforms.Resize((input_resolution, input_resolution)))
    transform_list.append(transforms.ToTensor())
    transform_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return transforms.Compose(transform_list)


def run():
    column_names = ['frame', 'left', 'top', 'right', 'bottom']
    df = pd.read_csv(args.head, names=column_names, index_col=0)
    df['left'] -= (df['right']-df['left'])*0.1
    df['right'] += (df['right']-df['left'])*0.1
    df['top'] -= (df['bottom']-df['top'])*0.1
    df['bottom'] += (df['bottom']-df['top'])*0.1

    # set up data transformation
    test_transforms = _get_transform()

    model = ModelSpatial()
    model_dict = model.state_dict()
    pretrained_dict = torch.load(args.model_weights)
    pretrained_dict = pretrained_dict['model']
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    model.cuda()
    model.train(False)
    df2=pd.DataFrame(columns=['x', 'y'])

    with torch.no_grad():
        for i in df.index:
            frame_raw = Image.open(os.path.join(args.image_dir, i))
            frame_raw = frame_raw.convert('RGB')
            width, height = frame_raw.size

            head_box = [df.loc[i,'left'], df.loc[i,'top'], df.loc[i,'right'], df.loc[i,'bottom']]

            head = frame_raw.crop((head_box)) # head crop

            head = test_transforms(head) # transform inputs
            frame = test_transforms(frame_raw)
            head_channel = imutils.get_head_box_channel(head_box[0], head_box[1], head_box[2], head_box[3], width, height,
                                                        resolution=input_resolution).unsqueeze(0)

            head = head.unsqueeze(0).cuda()
            frame = frame.unsqueeze(0).cuda()
            head_channel = head_channel.unsqueeze(0).cuda()

            # forward pass
            raw_hm, _, inout = model(frame, head_channel, head)

            # heatmap modulation
            raw_hm = raw_hm.cpu().detach().numpy() * 255
            raw_hm = raw_hm.squeeze()
            inout = inout.cpu().detach().numpy()
            inout = 1 / (1 + np.exp(-inout))
            inout = (1 - inout) * 255
            norm_map = resize(raw_hm, (height, width)) - inout

            # vis
            plt.close()
            fig = plt.figure()
            #fig.canvas.manager.window.move(0,0)
            plt.axis('off')
            plt.imshow(frame_raw)

            ax = plt.gca()
            rect = patches.Rectangle((head_box[0], head_box[1]), head_box[2]-head_box[0], head_box[3]-head_box[1], linewidth=2, edgecolor=(0,1,0), facecolor='none')
            ax.add_patch(rect)

            pred_x, pred_y = evaluation.argmax_pts(raw_hm)
            norm_p = [pred_x/output_resolution, pred_y/output_resolution]
            x, y = norm_p[0]*width, norm_p[1]*height
            row = {'x': x, 'y': y}
            #df2=df2.append(row, ignore_index=True)
            
            if args.vis_mode == 'arrow':
                if inout < args.out_threshold: # in-frame gaze
                    pred_x, pred_y = evaluation.argmax_pts(raw_hm)

                    norm_p = [pred_x/output_resolution, pred_y/output_resolution]
                    circ = patches.Circle((norm_p[0]*width, norm_p[1]*height), height/50.0, facecolor=(1,1,1), edgecolor='none')
                    ax.add_patch(circ)
                    plt.plot((norm_p[0]*width,(head_box[0]+head_box[2])/2), (norm_p[1]*height,(head_box[1]+head_box[3])/2), '-', color=(0,1,0,1))
            else:
                plt.imshow(norm_map, cmap = 'jet', alpha=0.2, vmin=0, vmax=255)
            

            plt.show(block=False)
            plt.pause(0.2)

        print('DONE!')
        df2.to_csv('person2.csv')
if __name__ == "__main__":
    run()


In [ ]:
import argparse, os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from skimage.transform import resize

from model import ModelSpatial
from utils import imutils, evaluation
from config import *

parser = argparse.ArgumentParser()
parser.add_argument('--model_weights', type=str, help='model weights', default='model_demo.pt')
parser.add_argument('--image_dir', type=str, help='images', default='data/demo/frames')
parser.add_argument('--head', type=str, help='head bounding boxes', default='data/demo/person3.txt')
parser.add_argument('--vis_mode', type=str, help='heatmap or arrow', default='arrow')
parser.add_argument('--out_threshold', type=int, help='out-of-frame target dicision threshold', default=100)
args, unknown = parser.parse_known_args()

In [ ]:
def _get_transform():
    transform_list = []
    transform_list.append(transforms.Resize((input_resolution, input_resolution)))
    transform_list.append(transforms.ToTensor())
    transform_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return transforms.Compose(transform_list)


def run():
    column_names = ['frame', 'left', 'top', 'right', 'bottom']
    df = pd.read_csv(args.head, names=column_names, index_col=0)
    df['left'] -= (df['right']-df['left'])*0.1
    df['right'] += (df['right']-df['left'])*0.1
    df['top'] -= (df['bottom']-df['top'])*0.1
    df['bottom'] += (df['bottom']-df['top'])*0.1

    # set up data transformation
    test_transforms = _get_transform()

    model = ModelSpatial()
    model_dict = model.state_dict()
    pretrained_dict = torch.load(args.model_weights)
    pretrained_dict = pretrained_dict['model']
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    model.cuda()
    model.train(False)
    df2=pd.DataFrame(columns=['x', 'y'])

    with torch.no_grad():
        for i in df.index:
            frame_raw = Image.open(os.path.join(args.image_dir, i))
            frame_raw = frame_raw.convert('RGB')
            width, height = frame_raw.size

            head_box = [df.loc[i,'left'], df.loc[i,'top'], df.loc[i,'right'], df.loc[i,'bottom']]

            head = frame_raw.crop((head_box)) # head crop

            head = test_transforms(head) # transform inputs
            frame = test_transforms(frame_raw)
            head_channel = imutils.get_head_box_channel(head_box[0], head_box[1], head_box[2], head_box[3], width, height,
                                                        resolution=input_resolution).unsqueeze(0)

            head = head.unsqueeze(0).cuda()
            frame = frame.unsqueeze(0).cuda()
            head_channel = head_channel.unsqueeze(0).cuda()

            # forward pass
            raw_hm, _, inout = model(frame, head_channel, head)

            # heatmap modulation
            raw_hm = raw_hm.cpu().detach().numpy() * 255
            raw_hm = raw_hm.squeeze()
            inout = inout.cpu().detach().numpy()
            inout = 1 / (1 + np.exp(-inout))
            inout = (1 - inout) * 255
            norm_map = resize(raw_hm, (height, width)) - inout

            # vis
            plt.close()
            fig = plt.figure()
            #fig.canvas.manager.window.move(0,0)
            plt.axis('off')
            plt.imshow(frame_raw)

            ax = plt.gca()
            rect = patches.Rectangle((head_box[0], head_box[1]), head_box[2]-head_box[0], head_box[3]-head_box[1], linewidth=2, edgecolor=(0,1,0), facecolor='none')
            ax.add_patch(rect)

            pred_x, pred_y = evaluation.argmax_pts(raw_hm)
            norm_p = [pred_x/output_resolution, pred_y/output_resolution]
            x, y = norm_p[0]*width, norm_p[1]*height
            row = {'x': x, 'y': y}
            #df2=df2.append(row, ignore_index=True)
            if args.vis_mode == 'arrow':
                if inout < args.out_threshold: # in-frame gaze
                    pred_x, pred_y = evaluation.argmax_pts(raw_hm)

                    norm_p = [pred_x/output_resolution, pred_y/output_resolution]
                    circ = patches.Circle((norm_p[0]*width, norm_p[1]*height), height/50.0, facecolor=(1,1,1), edgecolor='none')
                    ax.add_patch(circ)
                    plt.plot((norm_p[0]*width,(head_box[0]+head_box[2])/2), (norm_p[1]*height,(head_box[1]+head_box[3])/2), '-', color=(0,1,0,1))
            else:
                plt.imshow(norm_map, cmap = 'jet', alpha=0.2, vmin=0, vmax=255)
            

            plt.show(block=False)
            plt.pause(0.2)

        print('DONE!')
        df2.to_csv('person2.csv')
if __name__ == "__main__":
    run()


In [ ]:
colnames=['x1','y1'] 
person1 = pd.read_csv('person1.csv', names=colnames)
colnames=['x2','y2'] 
person2 = pd.read_csv('person2.csv', names=colnames)

In [ ]:
person1 = person1.iloc[1: , :]
person2 = person2.iloc[1: , :]



In [ ]:
person2

,x2,y2
0.0,860.0,663.75
1.0,860.0,663.75
2.0,840.0,675.0
3.0,840.0,663.75
4.0,840.0,663.75
...,...,...
281.0,860.0,686.25
282.0,860.0,686.25
283.0,860.0,686.25
284.0,860.0,686.25


In [ ]:
circle_intersect_detect= pd.concat([person1, person2], axis=1)
circle_intersect_detect

,x1,y1,x2,y2
0.0,680.0,663.75,860.0,663.75
1.0,680.0,663.75,860.0,663.75
2.0,680.0,663.75,840.0,675.0
3.0,680.0,663.75,840.0,663.75
4.0,680.0,663.75,840.0,663.75
...,...,...,...,...
281.0,800.0,675.0,860.0,686.25
282.0,780.0,663.75,860.0,686.25
283.0,800.0,675.0,860.0,686.25
284.0,780.0,663.75,860.0,686.25


In [ ]:
circle_intersect_detect=pd.DataFrame(circle_intersect_detect).astype(float)


In [ ]:
circle_intersect_detect

,x1,y1,x2,y2
0.0,680.0,663.75,860.0,663.75
1.0,680.0,663.75,860.0,663.75
2.0,680.0,663.75,840.0,675.00
3.0,680.0,663.75,840.0,663.75
4.0,680.0,663.75,840.0,663.75
...,...,...,...,...
281.0,800.0,675.00,860.0,686.25
282.0,780.0,663.75,860.0,686.25
283.0,800.0,675.00,860.0,686.25
284.0,780.0,663.75,860.0,686.25


In [ ]:
def circle(x1, y1, x2, y2, r1, r2): 
   
    distSq = (x1 - x2) * (x1 - x2) + (y1 - y2) * (y1 - y2);  
    radSumSq = (r1 + r2) * (r1 + r2);  
    if (distSq <= radSumSq): 
        return 1 
    elif (distSq > radSumSq): 
        return -1 
    

In [ ]:
t = circle(circle_intersect_detect.x1[0], circle_intersect_detect.y1[0], circle_intersect_detect.x2[0], circle_intersect_detect.y2[0], 100, 100)  
if (t == 1): 
    print("Circle intersect/touch to each other.")  
elif (t < 0): 
    print("Circle not touch to each other.")  


Circle intersect/touch to each other.


In [ ]:
aa=[]
for i in range(len(circle_intersect_detect)):
  t=circle(circle_intersect_detect.x1[i], circle_intersect_detect.y1[i], circle_intersect_detect.x2[i], circle_intersect_detect.y2[i], 100, 100)  
  aa.append(t)

In [ ]:
aa=pd.DataFrame(aa).astype(float)
circle_intersect_detect = pd.concat([circle_intersect_detect, aa], axis=1)


In [ ]:
circle_intersect_detect

,x1,y1,x2,y2,0
0.0,680.0,663.75,860.0,663.75,1.0
1.0,680.0,663.75,860.0,663.75,1.0
2.0,680.0,663.75,840.0,675.00,1.0
3.0,680.0,663.75,840.0,663.75,1.0
4.0,680.0,663.75,840.0,663.75,1.0
...,...,...,...,...,...
281.0,800.0,675.00,860.0,686.25,1.0
282.0,780.0,663.75,860.0,686.25,1.0
283.0,800.0,675.00,860.0,686.25,1.0
284.0,780.0,663.75,860.0,686.25,1.0


In [ ]:

circle_intersect_detect.columns=['x1','y1','x2','y2','intersect'] 

In [ ]:
circle_intersect_detect

,x1,y1,x2,y2,intersect
0.0,680.0,663.75,860.0,663.75,1.0
1.0,680.0,663.75,860.0,663.75,1.0
2.0,680.0,663.75,840.0,675.00,1.0
3.0,680.0,663.75,840.0,663.75,1.0
4.0,680.0,663.75,840.0,663.75,1.0
...,...,...,...,...,...
281.0,800.0,675.00,860.0,686.25,1.0
282.0,780.0,663.75,860.0,686.25,1.0
283.0,800.0,675.00,860.0,686.25,1.0
284.0,780.0,663.75,860.0,686.25,1.0


In [ ]:

dataframe1 = pd.read_csv("data/demo/person1.txt", names=['detele','a1','a2','a3','a4'] )
dataframe1 = dataframe1.drop('detele', axis=1)
  


In [ ]:
dataframe1

,a1,a2,a3,a4
0,371,43,626,396
1,369,46,627,396
2,371,46,628,394
3,372,44,629,395
4,374,44,630,396
...,...,...,...,...
281,370,44,698,390
282,372,43,698,398
283,373,43,699,399
284,370,39,698,401


In [ ]:
dataframe2 = pd.read_csv("data/demo/person2.txt", names=['detele','b1','b2','b3','b4'] )
dataframe2 = dataframe2.drop('detele', axis=1)
  

In [ ]:
dataframe2

,b1,b2,b3,b4
0,908,240,1141,531
1,908,246,1138,531
2,908,246,1139,532
3,908,247,1137,533
4,908,249,1138,531
...,...,...,...,...
281,909,247,1148,537
282,909,246,1146,538
283,909,244,1145,534
284,910,246,1140,536


In [ ]:
dataframe1.a4[0]

396

In [ ]:
for i in range(len(dataframe1)):
  if dataframe1.a4[i]<dataframe2.b4[i]:
    print("person1 is the caregiver")
  else:
    print("person2 is the caregiver")

person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver
person1 is the caregiver


In [ ]:
def checkOverlap(R, Xc, Yc, X1, Y1, X2, Y2):
 
    
    Xn = max(X1, min(Xc, X2))
    Yn = max(Y1, min(Yc, Y2))
     
   
    Dx = Xn - Xc
    Dy = Yn - Yc
    return (Dx**2 + Dy**2) <= R**2

In [ ]:
checkOverlap(100, 680.0, 663.75,908,240,1141,531 ).astype(int)

AttributeError: ignored

In [ ]:
reccircle_intersect_detect= pd.concat([dataframe2, person1], axis=1)


In [ ]:
reccircle_intersect_detect=reccircle_intersect_detect.astype(float)


In [ ]:
aa=[]
for i in range(len(reccircle_intersect_detect)):
  t=checkOverlap(100, reccircle_intersect_detect.x1[i], reccircle_intersect_detect.y1[i], reccircle_intersect_detect.b1[i], reccircle_intersect_detect.b2[i], reccircle_intersect_detect.b3[i], reccircle_intersect_detect.b4[i])  
  aa.append(t)
  

In [ ]:
aa=pd.DataFrame(aa).astype(float)
reccircle_intersect_detect = pd.concat([reccircle_intersect_detect, aa], axis=1)
aa

In [ ]:
reccircle_intersect_detect

,b1,b2,b3,b4,x1,y1,0
0.0,908.0,240.0,1141.0,531.0,680.0,663.75,0.0
1.0,908.0,246.0,1138.0,531.0,680.0,663.75,0.0
2.0,908.0,246.0,1139.0,532.0,680.0,663.75,0.0
3.0,908.0,247.0,1137.0,533.0,680.0,663.75,0.0
4.0,908.0,249.0,1138.0,531.0,680.0,663.75,0.0
...,...,...,...,...,...,...,...
281.0,909.0,247.0,1148.0,537.0,800.0,675.00,0.0
282.0,909.0,246.0,1146.0,538.0,780.0,663.75,0.0
283.0,909.0,244.0,1145.0,534.0,800.0,675.00,0.0
284.0,910.0,246.0,1140.0,536.0,780.0,663.75,0.0
